This notebook shows how to fine-tune a BERT model (from huggingface) for our dataset recognition task.

Note that internet is needed during the training phase (for downloading the bert-base-cased model). Internet can be turned off during prediction.

## Install packages

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
import json
f = open("./drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
json_data = json.load(f)
os.environ["KAGGLE_USERNAME"] = json_data["username"]
os.environ["KAGGLE_KEY"] = json_data["key"]

In [3]:
%cd ./drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/

/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative


In [4]:
!kaggle competitions download -c coleridgeinitiative-show-us-the-data

  0% 0.00/31.0k [00:00<?, ?B/s]
100% 31.0k/31.0k [00:00<00:00, 4.25MB/s]
  0% 0.00/25.6k [00:00<?, ?B/s]
100% 25.6k/25.6k [00:00<00:00, 3.55MB/s]
  0% 0.00/44.1k [00:00<?, ?B/s]
100% 44.1k/44.1k [00:00<00:00, 11.6MB/s]
  0% 0.00/90.4k [00:00<?, ?B/s]
100% 90.4k/90.4k [00:00<00:00, 12.6MB/s]
  0% 0.00/18.6k [00:00<?, ?B/s]
100% 18.6k/18.6k [00:00<00:00, 2.55MB/s]
  0% 0.00/38.1k [00:00<?, ?B/s]
100% 38.1k/38.1k [00:00<00:00, 4.99MB/s]
  0% 0.00/59.2k [00:00<?, ?B/s]
100% 59.2k/59.2k [00:00<00:00, 8.22MB/s]
  0% 0.00/28.1k [00:00<?, ?B/s]
100% 28.1k/28.1k [00:00<00:00, 3.96MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 5.65MB/s]
  0% 0.00/44.6k [00:00<?, ?B/s]
100% 44.6k/44.6k [00:00<00:00, 6.19MB/s]
  0% 0.00/28.6k [00:00<?, ?B/s]
100% 28.6k/28.6k [00:00<00:00, 4.02MB/s]
  0% 0.00/21.0k [00:00<?, ?B/s]
100% 21.0k/21.0k [00:00<00:00, 3.00MB/s]
  0% 0.00/55.7k [00:00<?, ?B/s]
100% 55.7k/55.7k [00:00<00:00, 7.71MB/s]
  0% 0.00/41.8k [00:00<?, ?B/s]
100% 41.8k/41.8k [0

In [6]:
!ls

0007f880-0a9b-492d-9a58-76eb0b0e0bd7.json
0008656f-0ba2-4632-8602-3017b44c2e90.json
000e04d6-d6ef-442f-b070-4309493221ba.json
000efc17-13d8-433d-8f62-a3932fe4f3b8.json
0010357a-6365-4e5f-b982-582e6d32c3ee.json
002203f0-1c57-4400-abc1-b783c4085743.json
00243b98-f868-45e4-9b83-0c346c7ecad5.json
00248da3-ac1d-48fa-a95e-cc88553f9583.json
0026563b-d5b3-417d-bd25-7656b97a044f.json
002b53ee-5b30-497f-a2fd-56885563918c.json
002cbf56-5158-4ec7-83fd-51fa7829bb13.json
002fdc24-9ee2-42b5-b051-373faca90c4e.json
0030f840-8505-49c0-9991-da0d4c6c9496.json
0035a1ba-6d1e-487b-bc3d-d49c5d64a3e9.json
003822e7-b394-4d05-a32b-7c55ae9dd2e8.json
0041a4e4-6144-4780-a2a2-787222417a01.json
0046e50c-6d19-4f6b-a7c5-4fe935d3b6f8.json
0047dc7b-dfc8-4b43-97c7-5bd056897e80.json
004b86d1-3056-4f78-95fe-fabb82272570.json
005c079d-ec3d-4aff-ad21-a636692a8b00.json
2100032a-7c33-4bff-97ef-690822c43466.json
2f392438-e215-4169-bebf-21ac4ff253e1.json
3f316b38-1a24-45a9-8d8c-4e05a42257c6.json
8e6996b4-ca08-4c0b-bed2-aaf07a4c6a

In [ ]:
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

# Import

In [ ]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

random.seed(123)
np.random.seed(456)

In [ ]:
# copy my_seqeval.py to the working directory because the input directory is non-writable
!cp /kaggle/input/coleridge-packages/my_seqeval.py ./

# Hyper-parameters

In [ ]:
MAX_LENGTH = 64 # max no. words for each sentence.
OVERLAP = 20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.

# Load data

In [ ]:
train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'

train = pd.read_csv(train_path)
train = train[:MAX_SAMPLE]
print(f'No. raw training rows: {len(train)}')

Group by publication, training labels should have the same form as expected output.

In [ ]:
train = train.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()

print(f'No. grouped training rows: {len(train)}')

In [ ]:
papers = {}
for paper_id in train['Id'].unique():
    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers[paper_id] = paper

# Transform data to NER format

In [ ]:
def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

def tag_sentence(sentence, labels): # requirement: both sentence and labels are already cleaned
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = ['O'] * len(sentence_words)
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = 'B'
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = 'I'

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = ['O'] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [ ]:
cnt_pos, cnt_neg = 0, 0 # number of sentences that contain/not contain labels
ner_data = []

pbar = tqdm(total=len(train))
for i, id, dataset_label in train[['Id', 'dataset_label']].itertuples():
    # paper
    paper = papers[id]
    
    # labels
    labels = dataset_label.split('|')
    labels = [clean_training_text(label) for label in labels]
    
    # sentences
    sentences = set([clean_training_text(sentence) for section in paper 
                 for sentence in section['text'].split('.') 
                ])
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    # positive sample
    for sentence in sentences:
        is_positive, tags = tag_sentence(sentence, labels)
        if is_positive:
            cnt_pos += 1
            ner_data.append(tags)
        elif any(word in sentence.lower() for word in ['data', 'study']): 
            ner_data.append(tags)
            cnt_neg += 1
    
    # process bar
    pbar.update(1)
    pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")

# shuffling
random.shuffle(ner_data)

write data to file.

In [ ]:
with open('train_ner.json', 'w') as f:
    for row in ner_data:
        words, nes = list(zip(*row))
        row_json = {'tokens' : words, 'tags' : nes}
        json.dump(row_json, f)
        f.write('\n')

# Fine-tune a BERT model for NER

In [ ]:
!python ../input/kaggle-ner-utils/kaggle_run_ner.py \
--model_name_or_path 'bert-base-cased' \
--train_file './train_ner.json' \
--validation_file './train_ner.json' \
--num_train_epochs 1 \
--per_device_train_batch_size 8 \
--per_device_eval_batch_size 8 \
--save_steps 15000 \
--output_dir './output' \
--report_to 'none' \
--seed 123 \
--do_train 

After the tuning finishes, we should find our model in './output'.